In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama2"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()


In [2]:
# Testing invoke
model.invoke("Tell me a joke")


"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [3]:
# Testing Parser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser
chain.invoke("Tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing. If you want to hear another one, just let me know!"

In [4]:
# load pdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("pdf/fastapi_workshop_linuxtips.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='README.md\nFastAPI Workshop\nCriando uma aplicação web com API usando FastAPI\nRequisitos\nComputador com Python 3.10\nDocker & docker-compose\nOu https://gitpod.io para um ambiente online\nUm editor de códigos como VSCode, Sublime, Vim, Micro (Nota: Eu usarei o Micro-\nEditor)\nimportante: Os comandos apresentados serão executados em um terminal Linux, se\nestiver no Windows recomendo usar o WSL, uma máquina virtual ou um container\nLinux, ou por conta própria adaptar os comandos necessários.\nAmbiente\nPrimeiro precisamos de um ambiente virtual para insta lar as dependencias do projeto.\npython -m venv .venv\nE ativaremos a virtualenv\n# Linux \nsource .venv/bin/activate \n# Windows Power Shell \n.\\venv\\Scripts\\activate.ps1\nVamos instalar ferramentas de produtividade neste ambiente e para isso vamos criar um\narquivo chamado requirements-dev.txt\nipython         # terminal \nipdb            # debugger \nsdb             # debugger remoto \npip-tools       #

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question in portuguese based on the context below.
If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="here is some context", question="Here is a question")

'\nAnswer the question in portuguese based on the context below.\nIf you can\'t answer the question, reply "I don\'t know".\n\nContext: here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

In [7]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [8]:
chain.invoke(
    {
        "context": "The name of the city is Chapecó",
        "question": "What's the city name?"
    }
)

' Ah, entendo! A cidade é Chapecó! 😊'

In [9]:
# pesquisar vector databases, pinecone
from langchain_community.vectorstores import  DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [13]:
retriever = vectorstore.as_retriever()
retriever.invoke("JWT")

[Document(page_content='version: \'3.9\' \nservices: \n  api: \n    build: \n      context: . \n      dockerfile: Dockerfile.dev \n    ports: \n      - "8000:8000" \n    environment: \n      PAMPS_DB__uri: "postgresql://postgres:postgres@db:5432/${PAMPS_DB:-pamps}" \n      PAMPS_DB__connect_args: "{}" \n    volumes: \n      - .:/home/app/api \n    depends_on: \n      - db \n    stdin_open: true \n    tty: true \n  db: \n    build: postgres \n    image: pamps_postgres-13-alpine-multi-user \n    volumes: \n      - $HOME/.postgres/pamps_db/data/postgresql:/var/lib/postgresql/data \n    ports: \n      - "5432:5432" \n    environment: \n      - POSTGRES_DBS=pamps, pamps_test \n      - POSTGRES_USER=postgres \n      - POSTGRES_PASSWORD=postgres\nO próximo passo é executar com\ndocker-compose up\nDeﬁnindo os models com Pydantic', metadata={'source': 'pdf/fastapi_workshop_linuxtips.pdf', 'page': 8}),
 Document(page_content='A API ﬁnal', metadata={'source': 'pdf/fastapi_workshop_linuxtips.pdf',

In [18]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

print(chain.invoke({"question": "O que é jwt?"}))

# posso fazer uma lista de questoes e fazer um for para as questões

JWT (JSON Web Token) is a compact, URL-safe means of representing claims to be transferred between two parties. The token is digitally signed and contains a payload that can be verified by the recipient. JWTs are widely used for authentication and authorization in web applications.

In the context of the tutorial, JWT is being used to authenticate API requests. When a user sends an HTTP request to the API, the server generates a JWT token based on the user's credentials (username and password). The JWT token is then sent back to the client in the response, along with any data that the client is authorized to access.

When the client makes a subsequent request to the API, it includes the JWT token in the Authorization header of the request. The server can then verify the token and determine whether the client is authorized to access the requested resource. If the token is valid, the server will grant the client access to the resource; otherwise, it will return an error response.

JWTs h

In [19]:
for s in chain.stream({"question": "What is the purpose of the coure?"}):
    print(s, end="", flush=True)

The course appears to be a tutorial on how to create a FastAPI application using Python, with a focus on authentication and user management. The code snippets in the PDF are likely from the course materials or exercises for students to practice and learn by doing.

The purpose of the course can vary depending on the context and audience, but here are some possible goals:

1. Teach Python programming: The course may be designed to teach students the basics of Python programming, including data structures, control flow, functions, and modules.
2. Introduce FastAPI: The course may focus on introducing students to FastAPI, a modern web framework for building APIs with Python. Students will learn how to create routes, handle requests and responses, and work with different data formats.
3. Implement authentication and user management: The course may cover the basics of authentication and user management using FastAPI's built-in features and third-party libraries. Students will learn how to c

In [ ]:
# chain.batch([{"question": q} for q in question_list])